# Use a large language model to summarize sticky notes

### Scenario
Imagine you are a team lead and you are taking your team through a reflection activity. Your team has used a mural to record their feedback.  Now, you need to summarize the mural contents for reporting results.

Notebook sections:

**Section A - Explore prompting large languge models**
- [Step 1: Set up IBM watsonx.ai foundation model Python library prerequisites](#step1)
- [Step 2: Create a function for prompting a model to identify top themes in messages](#step2)
- [Step 3: Create a function for prompting a model to classify a message](#step3)
- [Step 4: Create a function for prompting a model to summarize messages](#step4)

**Section B - Set up your mural**
- [Step 5: Set up MURAL prerequisites](#step5)
- [Step 6: Read feedback messages from your mural](#step6)

**Section C - Cluster sticky notes in the mural**
- [Step 7: Identify top themes in sticky notes](#step7)
- [Step 8: Classify sticky notes](#step8)
- [Step 9: Cluster sticky notes in the mural](#step9)

**Section D - Summarize sticky notes in a report**
- [Step 10: Summarize sticky note classes](#step10)
- [Step 11: Generate HTML report](#step11)

When you run this notebook, your mural will look something like the image below.  

And you'll have a report summarizing the contents of the mural: <a href="https://github.com/spackows/MURAL-API-Samples/blob/main/images/sample-18_llm-summary-report.pdf" target="_other">sample-18_llm-summary-report.pdf</a>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-18_llm-summary_01.png" width="47%" title="Image of a mural" style="float: left;" />

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-18_llm-summary_05.png" width="47%" title="Image of a report" style="float: right;" />


# Section A - Explore prompting large languge models

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-18_llm-summary_02.png" width="60%" title="Image of a mural" />

<a id="step1"></a>
## Step 1: Set up IBM watsonx.ai foundation model Python library prerequisites
Before you can prompt a foundation model in watsonx.ai, you must perform the following setup tasks:
- 1.1 Create an instance of the Watson Machine Learning service
- 1.2 Associate the Watson Machine Learning instance with the current project
- 1.3 Create an IBM Cloud API key
- 1.4 Create a credentials dictionary for Watson Machine learning
- 1.5 Look up the current project ID

### 1.1 Create an instance of the Watson Machine Learning service
If you don't already have an instance of the IBM Watson Machine Learning service, you can create an instance of the service from the IBM Cloud catalog: <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank">Watson Machine Learning service</a>

### 1.2 Associate an instance of the Watson Machine Learning service with the current project
The _current project_ is the project in which you are running this notebook.

If an instance of Watson Machine Learning is not already associated with the current project, follow the instructions in this topic to do so: <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html?context=wx&audience=wdp" target="_blank">Adding associated services to a project</a>

### 1.3 Create an IBM Cloud API key
Create an IBM Cloud API key by following these instruction: <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui#create_user_key" target="_blank">Creating an IBM Cloud API key</a>

Then paste your new IBM Cloud API key in the code cell below.

In [2]:
g_cloud_apikey = ""

### 1.4 Create a credentials dictionary for Watson Machine learning
See: [Authentication](https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication)

In [3]:
g_wml_credentials = { 
    "url"    : "https://us-south.ml.cloud.ibm.com", 
    "apikey" : g_cloud_apikey
}

### 1.5 Look up the current project ID
The _current project_ is the project in which you are running this notebook.  You can get the ID of the current project programmatically by running the following cell.

In [4]:
import os

g_project_id = os.environ["PROJECT_ID"]

<a id="step2"></a>
## Step 2: Create a function for prompting a model to identify top themes in messages
- 2.1 Experiment in Prompt Lab
- 2.2 Specify your selected model ID, prompt parameters, and prompt text template
- 2.3 Define a function to identify themes

### 2.1 Experiment in Prompt Lab 
The Prompt Lab in watsonx.ai is a graphical interface for experimenting with prompting foundation models.

Experiment in Prompt Lab to discover what works best:
- Which model returns ideal results
- What parameter settings (eg. decoding) work best
- What prompt text causes the model to respond the way you want

See: [Prompt Lab](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-prompt-lab.html?context=wx&audience=wdp)

### 2.2 Specify your selected model_id, prompt parameters, and prompt text template
In the following three cells, there are example model ID, prompt parameters, and a prompt text template you can use.

Replace any of these with values you discovered while experimenting in Prompt Lab.

See:
- [Models in watsonx.ai](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx&audience=wdp)
- [Prompt parameters](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-parameters.html?context=wx&audience=wdp)

In [5]:
g_themes_model_id = "meta-llama/llama-2-70b-chat"

In [6]:
g_themes_prompt_parameters = {
    "decoding_method" : "greedy",
    "min_new_tokens"  : 0,
    "max_new_tokens"  : 60,
    "stop_sequences"  : [ "\n\n" ]
}

**NOTE**

In the following example template, notice the use of `%s` as a placeholder for the messages in the sticky notes. 

If you replace this template with a prompt you discovered through your experiments in Prompt Lab, remember to include a placeholder for the messages.

In [7]:
g_themes_prompt_template = """The following meals fall into three broad themes:
- Spaghetti with meatballs
- Vegetarian sub
- Shrimp pad thai
- Fish fingers
- Falafel
- Steak and kidney pie

Three themes in the list of meals:
- Meat
- Vegetarian
- Seafood


The following animals fall into three broad themes:
- Cow
- Chicken
- Dog
- Giraffe
- Gerbil
- Elephant

Three themes in the list of animals:
- Pet
- Farm
- Wild


The following process feedback ideas fall into three broad themes:
%s

Think past what each idea mentions and consider larger patterns across the ideas.

Three themes that get at the essence of the list of process feedback ideas:
"""

def createThemesPromptText( messages_arr ):
    messages_str = "- " + "\n- ".join( messages_arr )
    return g_themes_prompt_template % ( messages_str )

In [61]:
g_test_messages_arr = [ 
    "We need to focus more on automated regression processes to improve quality",
    "Our development process should devote 1 week out of every 8 to only fixing bugs", 
    "Streamline our defect approval process because it's too time-consuming and tedious", 
    "Onboarding new team members takes ages because our processes are so complex",
    "Our processes are not documented.. one or two of our senior people are the only ones to really understand everything",
    "The defect process is so time-consuming that nobody reports bugs",
    "We should streamline our processes",
    "Managers need specific training on how to manage remote employees", 
    "Increase the manager-to-employee ratio, because currently managers don't have time for each person they manage",
    "Regularly move people to different managers - to benefit from different management styles",
    "I wish we gave managers more support so they don't have to learn everything the hard way - derailing peoples' careers along the way",
    "Assess managers based on how successfully their reports grow and improve and get promoted",
    "From a career growth perspective, my manager is totally absent.. She and I meet once a year for evaluations and otherwise I never see her!",
    "If you have a weak manager, your career will stagnate until you move to a different team or company",
    "When people need flexible hours or time off, don't require them to specify a reason, because everyone's life has different pressures, no one reason is more legitimate than another, and not having to disclose would reduce bias and discrimination",
    "Give everyone the ability to work from home with no questions asked, so no one has to explain personal details of their life (eg. physical or mental disability, burnout, or caring obligations)",
    "Establish a blanket rule that nobody is required to respond to emails outside regular working hours",
    "After age 50, growth and development opportunities dry up and disappear, this is age-ism",
    "We should explore the 4-day work week",
    "My parents took sabbaticals every few years, it was a tremendous growth opportunity for them and they brought reviewed energy and new ideas back to work with them when they returned.  We should implement opportunities for sabbaticals too.",
    "We need better support and understanding for parents juggling childcare"
]

g_themes_test_prompt_text = createThemesPromptText( g_test_messages_arr )

print( g_themes_test_prompt_text )

The following meals fall into three broad themes:
- Spaghetti with meatballs
- Vegetarian sub
- Shrimp pad thai
- Fish fingers
- Falafel
- Steak and kidney pie

Three themes in the list of meals:
- Meat
- Vegetarian
- Seafood


The following animals fall into three broad themes:
- Cow
- Chicken
- Dog
- Giraffe
- Gerbil
- Elephant

Three themes in the list of animals:
- Pet
- Farm
- Wild


The following process feedback ideas fall into three broad themes:
- We need to focus more on automated regression processes to improve quality
- Our development process should devote 1 week out of every 8 to only fixing bugs
- Streamline our defect approval process because it's too time-consuming and tedious
- Onboarding new team members takes ages because our processes are so complex
- Our processes are not documented.. one or two of our senior people are the only ones to really understand everything
- The defect process is so time-consuming that nobody reports bugs
- We should streamline our proces

### 2.3 Define a function to identify themes
You can prompt foundation models in IBM watsonx.ai programmatically using the foundation models Python library.

See:
- <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-python-lib.html?context=wx&audience=wdp" target="_blank">Introduction to the foundation models Python library</a>
- <a href="https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html" target="_blank">Foundation models Python library reference</a>

In [50]:
from ibm_watson_machine_learning.foundation_models import Model
import json
import re

def generate( model_id, prompt_parameters, prompt_text, b_debug=False ):
    model = Model( model_id, g_wml_credentials, prompt_parameters, g_project_id )
    raw_response = model.generate( prompt_text )
    if b_debug:
        print( "\nraw_response:\n" + json.dumps( raw_response, indent=3 ) )
    if ( "results" in raw_response ) \
       and ( len( raw_response["results"] ) > 0 ) \
       and ( "generated_text" in raw_response["results"][0] ):
        return raw_response, raw_response["results"][0]["generated_text"]
    else:
        print( "\nThe model failed to generate an answer" )
        print( "\nDebug info:\n" + json.dumps( raw_response, indent=3 ) )
        return raw_response, ""

def identifyThemes( model_id, prompt_parameters, messages_arr, b_debug=False ):
    prompt_text = createThemesPromptText( messages_arr )
    raw_response, generated_output = generate( model_id, prompt_parameters, prompt_text, b_debug )
    class_names_str = re.sub( r"^\s*\-\s*", "", generated_output )
    class_names_str = re.sub( r"\s+$", "", class_names_str )
    class_names_arr = re.split( r"\s*\n+\-\s*", class_names_str )
    class_names_arr = [ item.strip().capitalize() for item in class_names_arr ]
    return class_names_arr

In [51]:
g_test_themes_arr = identifyThemes( g_themes_model_id, g_themes_prompt_parameters, g_test_messages_arr, b_debug=True )

print( "\ng_test_themes_arr:\n" + json.dumps( g_test_themes_arr, indent=3 ) )


raw_response:
{
   "model_id": "meta-llama/llama-2-70b-chat",
   "created_at": "2023-11-18T23:03:07.070Z",
   "results": [
      {
         "generated_text": "- Improving employee growth and development\n- Work-life balance and flexibility\n- Streamlining and optimizing processes\n\n",
         "generated_token_count": 28,
         "input_token_count": 696,
         "stop_reason": "stop_sequence"
      }
   ],
   "system": {
      "warnings": [
         {
            "message": "This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL. URL: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx",
            "id": "disclaimer_warning"
         }
      ]
   }
}

g_test_themes_arr:
[
   "Improving employee growth and development",
   "Work-life balance and flexibility",
   "Streamlining and optimizing processes

<a id="step3"></a>
## Step 3: Create a function for prompting a model to classify messages
- 3.1 Experiment in Prompt Lab again
- 3.2 Specify your selected model ID, prompt parameters, and prompt text template
- 3.3 Define a function to perform classification

### 3.1 Experiment in Prompt Lab again
Experiment in Prompt Lab to discover what works best:
- Which model returns ideal results
- What parameter settings (eg. decoding) work best
- What prompt text causes the model to respond the way you want

### 3.2 Specify your selected model_id, prompt parameters, and prompt text template
In the following three cells, there are example model ID, prompt parameters, and a prompt text template you can use.

Replace any of these with values you discovered while experimenting in Prompt Lab.

In [10]:
g_classify_model_id = "google/flan-t5-xxl"

In [11]:
g_classify_prompt_parameters = {
    "decoding_method" : "greedy",
    "min_new_tokens"  : 0,
    "max_new_tokens"  : 20
}

**NOTE**

In the following example template, notice the use of `%s` as a placeholder for the class names and message text. 

If you replace this template with a prompt you discovered through your experiments in Prompt Lab, remember to include a placeholder for the class names and one for the message text.

In [53]:
g_classify_prompt_template = """Classify the message into one of three classes: %s

If the message doesn't fit any of the classes, say "Other"

Message: %s
Class: 
"""

def createClassifyPromptText( class_names_arr, message_text ):
    class_name_str = ", ".join( class_names_arr )
    return g_classify_prompt_template % ( class_name_str, message_text )

In [62]:
g_classify_test_message = "I think we should record our meetings so they could be played back"

g_classify_test_prompt_text = createClassifyPromptText( g_test_themes_arr, g_classify_test_message )

print( g_classify_test_prompt_text )

Classify the message into one of three classes: Improving employee growth and development, Work-life balance and flexibility, Streamlining and optimizing processes

If the message doesn't fit any of the classes, say "Other"

Message: I think we should record our meetings so they could be played back
Class: 



### 3.3 Define a function to perform classification

In [56]:
def classifyMessage( model_id, prompt_parameters, class_names_arr, message_text, b_debug=False ):
    prompt_text = createClassifyPromptText( class_names_arr, message_text )
    raw_response, generated_output = generate( model_id, prompt_parameters, prompt_text, b_debug )
    class_name = generated_output.strip().capitalize()
    return class_name

In [57]:
g_summarize_test_class_name = classifyMessage( g_classify_model_id, g_classify_prompt_parameters, g_test_themes_arr, g_classify_test_message, b_debug=True )

print( "\ng_summarize_test_class_name: " + g_summarize_test_class_name )


raw_response:
{
   "model_id": "google/flan-t5-xxl",
   "created_at": "2023-11-18T23:04:58.143Z",
   "results": [
      {
         "generated_text": "Streamlining and optimizing processes",
         "generated_token_count": 8,
         "input_token_count": 69,
         "stop_reason": "eos_token"
      }
   ],
   "system": {
      "warnings": [
         {
            "message": "This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL. URL: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx",
            "id": "disclaimer_warning"
         }
      ]
   }
}

g_summarize_test_class_name: Streamlining and optimizing processes


<a id="step4"></a>
## Step 4: Create a function for prompting a model to summarize messages
- 4.1 Experiment in Prompt Lab again
- 4.2 Specify your selected model ID, prompt parameters, and prompt text template
- 4.3 Define a function to perform summarization

### 4.1 Experiment in Prompt Lab again
Experiment in Prompt Lab to discover what works best:
- Which model returns ideal results
- What parameter settings (eg. decoding) work best
- What prompt text causes the model to respond the way you want

### 4.2 Specify your selected model_id, prompt parameters, and prompt text template
In the following three cells, there are example model ID, prompt parameters, and a prompt text template you can use.

Replace any of these with values you discovered while experimenting in Prompt Lab.

In [15]:
g_summarize_model_id = "meta-llama/llama-2-70b-chat"

In [16]:
g_summarize_prompt_parameters = {
    "decoding_method" : "sample",
    "random_seed": 2788563498,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 1,
    "repetition_penalty": 1,
    "stop_sequences": [ "\n\n" ],
    "min_new_tokens"  : 0,
    "max_new_tokens"  : 240
}

**NOTE**

In the following example template, notice the use of `%s` as a placeholder for the class name and a list of messages. 

If you replace this template with a prompt you discovered through your experiments in Prompt Lab, remember to include a placeholder for the class name and one for a list of messages.

In [17]:
g_summarize_prompt_template = """You are a management consultant hired to advice business leaders based on input from employees.

The following employee input is related to %s:
%s

Write a one paragraph recommendation to company leaders based on only the employee input.  Take into account as much feedback as possible.  Identify common patterns across ideas.  Reflect the underlying human factors and needs of the employee input.  Explain how addressing these needs will benefit the company's success and bottom line.  Do not suggest ideas that are not listed in the employee ideas.  Use passive voice and do not refer to yourself; for example, say "it is recommended" instead of "we recommend".

Recommendation:
"""

def createSummarizePromptText( class_name, messages_arr ):
    messages_str = "- " + "\n- ".join( messages_arr )
    return g_summarize_prompt_template % ( class_name.lower(), messages_str )

In [63]:
g_summarize_test_messages_arr = [
    "We need to focus more on automated regression processes to improve quality",
    "Our development process should devote 1 week out of every 8 to only fixing bugs",
    "Streamline our defect approval process because it's too time-consuming and tedious",
    "Onboarding new team members takes ages because our processes are so complex",
    "Our processes are not documented.. one or two of our senior people are the only ones to really understand everything",
    "The defect process is so time-consuming that nobody reports bugs",
    "We should streamline our processes"
]

g_summarize_test_prompt_text = createSummarizePromptText( g_summarize_test_class_name, g_summarize_test_messages_arr )

print( g_summarize_test_prompt_text )

You are a management consultant hired to advice business leaders based on input from employees.

The following employee input is related to streamlining and optimizing processes:
- We need to focus more on automated regression processes to improve quality
- Our development process should devote 1 week out of every 8 to only fixing bugs
- Streamline our defect approval process because it's too time-consuming and tedious
- Onboarding new team members takes ages because our processes are so complex
- Our processes are not documented.. one or two of our senior people are the only ones to really understand everything
- The defect process is so time-consuming that nobody reports bugs
- We should streamline our processes

Write a one paragraph recommendation to company leaders based on only the employee input.  Take into account as much feedback as possible.  Identify common patterns across ideas.  Reflect the underlying human factors and needs of the employee input.  Explain how addressing t

### 4.3 Define a function to perform summarization

In [59]:
def summarizeMessages( model_id, prompt_parameters, class_name, messages_arr, b_debug=False ):
    prompt_text = createSummarizePromptText( class_name, messages_arr )
    raw_response, generated_output = generate( model_id, prompt_parameters, prompt_text, b_debug )
    return generated_output.strip()

In [60]:
summarizeMessages( g_summarize_model_id, g_summarize_prompt_parameters, g_summarize_test_class_name, g_summarize_test_messages_arr, b_debug=True )


raw_response:
{
   "model_id": "meta-llama/llama-2-70b-chat",
   "created_at": "2023-11-18T23:06:38.487Z",
   "results": [
      {
         "generated_text": "\nIt is recommended that the organization prioritize the optimization of processes to enhance efficiency, quality, and employee satisfaction. This can be achieved by implementing automated regression processes, dedicating specific periods for bug fixing, and simplifying the defect approval process. Moreover, streamlining the onboarding process and documenting processes can help reduce complexity and improve understanding among team members. Addressing these needs will not only improve the quality of work but also increase employee morale, productivity, and ultimately, the company's bottom line. By creating a more efficient and effective work environment, the organization can reduce the time spent on tedious tasks, allowing employees to focus on high-value activities that contribute to the company's success.",
         "generated

"It is recommended that the organization prioritize the optimization of processes to enhance efficiency, quality, and employee satisfaction. This can be achieved by implementing automated regression processes, dedicating specific periods for bug fixing, and simplifying the defect approval process. Moreover, streamlining the onboarding process and documenting processes can help reduce complexity and improve understanding among team members. Addressing these needs will not only improve the quality of work but also increase employee morale, productivity, and ultimately, the company's bottom line. By creating a more efficient and effective work environment, the organization can reduce the time spent on tedious tasks, allowing employees to focus on high-value activities that contribute to the company's success."

# Section B - Set up your mural

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-18_llm-summary_03.png" width="50%" title="Image of a mural" />

<a id="step"></a>
## Step 5: Set up MURAL prerequisites
- 5.1 Create empty, sample mural
- 5.2 Collect mural ID
- 5.3 Get Oauth token
- 5.4 Load helper functions
- 5.5 Populate mural

### 5.1 Create empty, sample mural
In the MURAL web interface, create a new, empty mural.

### 5.2 Collect mural ID
You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [20]:
g_mural_id = ""

### 5.3 Collect OAuth token

In [21]:
g_auth_token = ""

### 5.4 Load helper functions

To simplify this notebook, some useful, MURAL-related functions are implemented in a supporting file: [sample-18_llm-summary.py](https://github.com/spackows/MURAL-API-Samples/blob/main/helper-functions/sample-18_llm-summary.py)

Helpful functions:
- putWidget


In [22]:
from requests import get

url = "https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/helper-functions/sample-18_llm-summary.py"

local_file_name = "helper_functions.py"
with open( local_file_name, "wb") as file:
    response = get( url )
    file.write( response.content )

import helper_functions as hf

### 5.5 Populate mural

In [23]:
import urllib

url = "https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/murals/sample-18_llm-summary.json"
response = urllib.request.urlopen( url )
encoding = response.info().get_content_charset( "utf8" )
sample_widgets_arr = json.loads( response.read().decode( encoding ) )

In [24]:
import time

time.sleep(5)

# Quick!  After clicking Run on this cell, switch to 
# your browser tab where the mural is to see the change
# ...

error_str = hf.putWidgets( g_auth_token, g_mural_id, sample_widgets_arr )

if error_str:
    print( error_str )
else:
    print( "Done!" )

Done!


<a id="step5"></a>
## Step 6: Read feedback from your mural

In [25]:
error_str, g_widgets_arr = hf.listWidgets( g_auth_token, g_mural_id )

if error_str:
    print( error_str )
else:
    print( "Done!" )

Done!


In [ ]:
g_stickies_arr = []
for widget in g_widgets_arr:
    sticky = { "id"     : widget["id"],
               "org_x"  : widget["x"],
               "org_y"  : widget["y"],
               "height" : widget["height"],
               "width"  : widget["width"],
               "text"   : widget["text"] }
    g_stickies_arr.append( sticky )
    
g_stickies_arr

# Section C - Summarize sticky notes

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-18_llm-summary_01.png" width="50%" title="Image of a mural" />

<a id="step6"></a>
## Step 7: Identify top themes in sticky notes

In [27]:
def identifyThemesInStickies( model_id, prompt_parameters, stickies_arr, b_debug=False ):
    messages_arr = []
    for sticky in stickies_arr:
        messages_arr.append( sticky["text"] )
    class_names_arr = identifyThemes( model_id, prompt_parameters, g_test_messages_arr, b_debug )
    return class_names_arr

In [28]:
g_class_names_arr = identifyThemesInStickies( g_themes_model_id, g_themes_prompt_parameters, g_stickies_arr )

print( json.dumps( g_class_names_arr, indent=3 ) )

[
   "Improving employee growth and development",
   "Work-life balance and flexibility",
   "Streamlining and optimizing processes"
]


<a id="step8"></a>
## Step 8: Classify sticky notes

In [29]:
def classifyStickies( class_names_arr, stickies_arr ):
    classified_stickies = {}
    for sticky in stickies_arr:
        class_name = classifyMessage( g_classify_model_id, g_classify_prompt_parameters, class_names_arr, sticky["text"] )
        if( class_name not in classified_stickies ):
            classified_stickies[ class_name ] = []
        classified_stickies[ class_name ].append( sticky )
    return classified_stickies

In [30]:
g_classified_stickies = classifyStickies( g_class_names_arr, g_stickies_arr )

In [44]:
g_classifications = {}
for class_name in g_classified_stickies.keys():
    print( "\n" + class_name )
    g_classifications[ class_name ] = []
    for sticky in g_classified_stickies[ class_name ]:
        print( "- " + sticky["text"][0:80] )
        g_classifications[ class_name ].append( sticky["text"] )


Streamlining and optimizing processes
- We need to focus more on automated regression processes to improve quality
- Our development process should devote 1 week out of every 8 to only fixing bugs
- Streamline our defect approval process because it's too time-consuming and tedio
- Onboarding new team members takes ages because our processes are so complex
- Our processes are not documented.. one or two of our senior people are the only 
- The defect process is so time-consuming that nobody reports bugs
- We should streamline our processes

Work-life balance and flexibility
- Managers need specific training on how to manage remote employees
- When people need flexible hours or time off, don't require them to specify a rea
- Give everyone the ability to work from home with no questions asked, so no one h
- Establish a blanket rule that nobody is required to respond to emails outside re
- We should explore the 4-day work week
- We need better support and understanding for parents jugglin

<a id="step9"></a>
## Step 9: Cluster sticky notes in the mural
- 9.1 Create pinwheel with a pie slice for each class
- 9.2 Label each pie slice
- 9.3 Move sticky notes into the appropriate slide by class

### 9.1 Create pinwheel with a pie slice for each class

In [32]:
g_arcs_arr = hf.createArcs( g_class_names_arr )

g_arcs_arr

[{'class_name': 'Improving employee growth and development',
  'start_angle': 30.0,
  'end_angle': 150.0},
 {'class_name': 'Work-life balance and flexibility',
  'start_angle': 150.0,
  'end_angle': 270.0},
 {'class_name': 'Streamlining and optimizing processes',
  'start_angle': 270.0,
  'end_angle': 390.0}]

In [33]:
time.sleep(5)

# Quick!  After clicking Run on this cell, switch to 
# your browser tab where the mural is to see the change
# ...

error_str = hf.putArcs( g_auth_token, g_mural_id, g_arcs_arr )

if error_str:
    print( error_str )
else:
    print( "Done!" )

Done!


### 9.2 Label each pie slice

In [34]:
time.sleep(5)

# Quick!  After clicking Run on this cell, switch to 
# your browser tab where the mural is to see the change
# ...

error_str = hf.putClassNames( g_auth_token, g_mural_id, g_arcs_arr )

if error_str:
    print( error_str )
else:
    print( "Done!" )

Done!


### 9.3 Move sticky notes into the appropriate slice by class

In [35]:
time.sleep(5)

# Quick!  After clicking Run on this cell, switch to 
# your browser tab where the mural is to see the change
# ...

error_str = hf.moveStickiesToClassArcs( g_auth_token, g_mural_id, g_arcs_arr, g_classified_stickies )

if error_str:
    print( error_str )
else:
    print( "Done!" )

Done!


Something like this:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-18_llm-summary_04.gif" width="50%" title="Image of a mural" />

<a id="step10"></a>
## Step 10: Summarize sticky note classes

In [36]:
def summarizeStickies( classified_stickies ):
    summaries = {}
    for class_name in classified_stickies.keys():
        messages_arr = []
        for sticky in classified_stickies[ class_name ]:
            messages_arr.append( sticky["text"] )
        summary = summarizeMessages( g_summarize_model_id, g_summarize_prompt_parameters, class_name, messages_arr )
        summaries[ class_name ] = summary
    return summaries

In [37]:
g_summaries = summarizeStickies( g_classified_stickies )

g_summaries

{'Streamlining and optimizing processes': "It is recommended that the organization prioritize the optimization of processes to enhance efficiency, quality, and employee satisfaction. This can be achieved by implementing automated regression processes, dedicating specific periods for bug fixing, and simplifying the defect approval process. Moreover, streamlining the onboarding process and documenting processes can help reduce complexity and improve understanding among team members. Addressing these needs will not only improve the quality of work but also increase employee morale, productivity, and ultimately, the company's bottom line. By creating a more efficient and effective work environment, the organization can reduce the time spent on tedious tasks, allowing employees to focus on high-value activities that contribute to the company's success.",
 'Work-life balance and flexibility': "It is recommended that the company provide more flexible work arrangements and support for employee

<a id="step11"></a>
## Step 11: Generate HTML report

In [68]:
g_llm_highlight_begin = "<mark style=\"background-color: cream; padding: 1px 3px 1px 3px;\">"
g_llm_highlight_end = "</mark>"

g_emp_highlight_begin = "<mark style=\"background-color: #FFE6E8; padding: 1px 3px 1px 3px;\">"
g_emp_highlight_end = "</mark>"


def reportIntro():
    
    report_html = """
<h1>Employee input report</h1>

<div style="border: 1px solid lightgrey; margin-top: 30px; padding: 0px 20px 20px 20px;">
<h2>About this report</h2>
<p>This report was compiled automatically and includes content from multiple sources:</p>
<ul>
<li>Verbatim comments from employees (<mark style=\"background-color: #FFE6E8; padding: 1px 3px 1px 3px;\">highlighted pink</mark>)</li>
<li>Content that was generated by a large language model (<mark style=\"background-color: cream; padding: 1px 3px 1px 3px;\">highlighted yellow</mark>)</li>
</ul>
<p>See appendices A and B for details.</p>
</div>

"""
    return report_html


def reportThemes( class_names_arr ):
    
    report_html = "<h2>Themes</h2>\n" + \
                  "<p>The following themes emerged from the employee input:</p>\n" + \
                  "<ul>\n"
    
    for class_name in class_names_arr:
        report_html += "<li>" + g_llm_highlight_begin + class_name + g_llm_highlight_end + "</li>\n"
    
    report_html += "</ul>\n\n"
    
    return report_html


def recommendationSummaries( class_names_arr, summaries ):
    
    report_html = "<h2>Recommendations by theme</h2>\n" + \
                  "<dl>\n"
    
    for class_name in class_names_arr:
        report_html += "<dt><br/>" + g_llm_highlight_begin + class_name + g_llm_highlight_end + "</dt>\n" + \
                       "<dd>" + g_llm_highlight_begin + summaries[ class_name ] + g_llm_highlight_end + "</dd>\n"
    
    report_html += "</dl>\n\n"
    
    return report_html


def appendixA( class_names_arr, classifications ):
    
    report_html = "<h2>Appendix A: Raw feedback messages</h2>\n"
    
    for class_name in class_names_arr:
        
        report_html += "<h3>" + g_llm_highlight_begin + class_name + g_llm_highlight_end + "</h3>\n" + \
                       "<ul>\n"
        
        for message in classifications[ class_name ]:
            report_html += "<li>" + g_emp_highlight_begin + message + g_emp_highlight_end + "</li>\n"
            
        report_html += "</ul>\n"
        
    report_html += "\n"
    
    return report_html


def appendixBIntro():
    
    report_html = "<h2>Appendix B: Generated content</h2>\n" + \
                  "<p>Parts of this report were generated using large language models hosted in: " + \
                  "<a href=\"https://www.ibm.com/products/watsonx-ai\">IBM watsonx.ai</a></p>\n"

    return report_html

    
def appendixBThemes( model_id, model_info_url, prompt_parameters, prompt_text ):
    
    report_html = "<h3>Themes</h3>\n" + \
                  "<p>Themes were generated using the following prompt details:</p>\n" + \
                  "<ul>\n" + \
                  "<li><b>Model</b> : <code>" + model_id + "</code> " + \
                      "(See: <a href=\"" + model_info_url + "\">Model details</a>)<br/>&nbsp;</li>\n" + \
                  "<li><b>Parameters</b> : <pre>" + \
                  json.dumps( prompt_parameters, indent=3 ) + \
                  "</pre><br/></li>\n" + \
                  "<li><b>Prompt text</b>\n" + \
                  "<div style=\"background: whitesmoke;  margin-top: 14px; padding: 5px;\">\n" + \
                  re.sub( r"\n", "<br/>", prompt_text ) + "\n" + \
                  "</div>\n" + \
                  "</ul>\n\n"

    return report_html


def appendixBClassification( model_id, model_info_url, prompt_parameters, prompt_text ):
    
    report_html = "<h3>Classification</h3>\n" + \
                  "<p>Employee input messages were classified by theme using the following prompt details:\n" + \
                  "<ul>\n" + \
                  "<li><b>Model</b> : <code>" + model_id + "</code> " + \
                  "(See: <a href=\"" + model_info_url + "\">Model details</a>)<br/>&nbsp;</li>\n" + \
                  "<li><b>Parameters</b> : <pre>" + \
                  json.dumps( prompt_parameters, indent=3 ) + \
                  "</pre><br/></li>\n" + \
                  "<li><b>Prompt text</b><br/>\n" + \
                  "(The comma-separated list of theme names was substituted into the prompt as " + \
                  "well as each message, one at a time, to be classified.)\n" + \
                  "<div style=\"background: whitesmoke;  margin-top: 14px; padding: 5px;\">\n" + \
                  re.sub( r"\n", "<br/>", prompt_text ) + "\n" + \
                  "</div>\n" + \
                  "</ul>\n\n"

    return report_html


def appendixBSummary( model_id, model_info_url, prompt_parameters, prompt_text ):
    
    report_html = "<h3>Summarization</h3>\n" + \
                  "<p>Employee input messages in each theme were summarized using the following prompt details:\n" + \
                  "<ul>\n" + \
                  "<li><b>Model</b> : <code>" + model_id + "</code> " + \
                  "(See: <a href=\"" + model_info_url + "\">Model details</a>)<br/>&nbsp;</li>\n" + \
                  "<li><b>Parameters</b> : <pre>" + \
                  json.dumps( prompt_parameters, indent=3 ) + \
                  "</pre><br/></li>\n" + \
                  "<li><b>Prompt text</b><br/>\n" + \
                  "(The following prompt was run for each theme.  Each time, the theme and message " + \
                  "classified as belonging to that them were substituted in the prompt.)\n" + \
                  "<div style=\"background: whitesmoke; margin-top: 14px; padding: 5px;\">\n" + \
                  re.sub( r"\n", "<br/>", prompt_text ) + "\n" + \
                  "</div>\n" + \
                  "</ul>\n\n"
    
    return report_html

In [72]:
g_llama_model_info_url = "https://huggingface.co/meta-llama/Llama-2-70b-chat"
g_flant5_model_info_url = "https://huggingface.co/google/flan-t5-xxl"

html = reportIntro() + \
       reportThemes( g_class_names_arr ) + \
       recommendationSummaries( g_class_names_arr, g_summaries ) + \
       appendixA( g_class_names_arr, g_classifications ) + \
       appendixBIntro() + \
       appendixBThemes( g_themes_model_id, g_llama_model_info_url, g_themes_prompt_parameters, g_themes_test_prompt_text ) + \
       appendixBClassification( g_classify_model_id, g_flant5_model_info_url, g_classify_prompt_parameters, g_classify_test_prompt_text ) + \
       appendixBSummary( g_summarize_model_id, g_llama_model_info_url, g_summarize_prompt_parameters, g_summarize_test_prompt_text )

In [95]:
from IPython.display import display, HTML

display( HTML( html ) )